# PREFACE

## Identity

======================================

Kisi - Kisi Live Code 2

Nama    : Rafina Dhiya Pradani

Batch   : HCK-22

======================================

# ANSWER

## Authenticate BigQuery

In [37]:
# Libraries
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Untuk mengotentikasi bigquery - colab
auth.authenticate_user()

# project_id
client = bigquery.Client(project="afi-ftds-hacktiv8")

### Melihat Data Frame Secara Keseluruhan

In [38]:
df_all = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.practice.bike_rent`
''').to_dataframe()

df_all

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60
...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55


### A.) SQL

#### 1. Berapa banyak label unique dari bike_type?

In [39]:
# DISTINCT digunakan untuk mengambil unique value dari kolom 'bike_type'
# COUNT digunakan untuk menghitung total unique value dari kolom 'bike_type'
df_1 = client.query('''
SELECT
  COUNT(DISTINCT bike_type) AS total_unique_bike_type
FROM `ftds-hacktiv8-project.practice.bike_rent`
''').to_dataframe()

# Menampilkan df_1
df_1

,total_unique_bike_type
0,2


Terdapat **2 unique value** pada `bike_type`

#### 2. Apa saja nama label unique dari tipe sepeda?

In [40]:
# DISTINCT digunakan untuk mengambil unique value dari kolom 'bike_type'
df_2 = client.query('''
SELECT
  DISTINCT bike_type AS unique_bike_type
FROM `ftds-hacktiv8-project.practice.bike_rent`
''').to_dataframe()

# Menampilkan df_2
df_2

,unique_bike_type
0,classic
1,electric


Unique value pada tipe sepeda terdiri dari **classic** dan **electric**

#### 3. Berapa banyak total peminjaman sepeda pada tahun 2016 dan 2017?

In [41]:
# EXTRACT digunakan untuk mengekstrak tahun (2016 dan 2017) dari kolom 'start_time'
df_3 = client.query("""
SELECT
  COUNT(*) AS total_rent
FROM
  `ftds-hacktiv8-project.practice.bike_rent`
WHERE
  EXTRACT(YEAR FROM start_time) IN (2016, 2017);
""").to_dataframe()

# Memanggil df_3
df_3

,total_rent
0,7993


Terdapat **7993 total peminjaman sepeda** pada tahun 2016 hingga 2018

#### 4. Dimana station yang paling banyak dikunjungi sebagai titik keberangkatan pada tahun 2016?

In [42]:
# 1. Memilih kolom start_station_name & menghitung jumlah total perjalanan untuk semua station. Hasil perhitungan disimpan dalam kolom most_visited_station
# 2. Memfilter data hanya untuk tahun 2016 dengan mengekstrak tahun dari kolom 'start_time'
# 3. Data dikelompokkan berdasarkan 'start_station_name
# 4. 'most_visited'station' diurutkan secara descending (besar ke kecil)
# 5. Limit = 1 maksudnya hanya mengambil 1 baris hasil, berupa stasiun yang paling banyak dikunjungi pada tahun 2016

df_4 = client.query("""
SELECT
  start_station_name,
  COUNT(*) AS most_visited_station
FROM
  `ftds-hacktiv8-project.practice.bike_rent`
WHERE
  EXTRACT(YEAR FROM start_time) IN (2016)
GROUP BY
  start_station_name
ORDER BY
  most_visited_station DESC
LIMIT 1
""").to_dataframe();

# Memanggil df_4
df_4

,start_station_name,most_visited_station
0,Bullock Museum @ Congress & MLK,394


Stasiun **Bullock Museum @ Congress & MLK** merupakan stasiun yang paling sering dikunjungi sebagai titik keberangkatan pada tahun 2016, yaitu dikunjungi sebanyak **394** kali.

#### 5. Berapa nilai minimum, maximum, dan rata-rata dari durasi perjalanan pada keseluruhan data?

In [43]:
# Mencari nilai minimum, maximum, dan rata-rata dari kolom 'duration_minutes'
df_5 = client.query("""
SELECT
  MIN(duration_minutes) AS minimum_duration,
  MAX(duration_minutes) AS maximum_duration,
  AVG(duration_minutes) AS average_duration
FROM
  `ftds-hacktiv8-project.practice.bike_rent`
""").to_dataframe();

# Memanggil df_5
df_5

,minimum_duration,maximum_duration,average_duration
0,2,2218,35.0995


Dari tahun 2016 hingga 2018:

*   Peminjaman sepeda yang paling singkat adalah selama **2 menit**
*   Peminjaman sepeda paling lama adalah selama **2,218 menit**
*   Durasi rata-rata para customer meminjam sepeda adalah selama **35.09 menit**


#### 6. Berapa rata-rata durasi perjalanan dari subscriber_type `U.T. Student Membership` dan `Local30`?

In [44]:
# 1. Memilih kolom 'subscriber_type' untuk mengelompokkan data berdasarkan jenis pelanggan
# 2. Menghitung rata-rata durasi perjalanan untuk setiap jenis pelanggan
# 3. Filter data hanya untuk jenis pelanggan U.T. Student Membership dan 'Local30
# 4. Mengelompokkan data berdasarkan subscriber type
df_6 = client.query("""
SELECT
  subscriber_type,
  AVG(duration_minutes) AS average_duration
FROM
  `ftds-hacktiv8-project.practice.bike_rent`
WHERE
  subscriber_type IN ('U.T. Student Membership', 'Local30')
GROUP BY
  subscriber_type
""").to_dataframe();

# Memanggil df_6
df_6

,subscriber_type,average_duration
0,Local30,16.021672
1,U.T. Student Membership,39.000000


*   Peminjam sepeda dengan tipe `Local30` rata-rata meminjam sepeda selama **16.02** menit pada tahun 2016 hingga 2018
*   Peminjam sepeda dengan tipe `U.T. Student Membership Student Membership` rata-rata meminjam sepeda selama **39 menit** pada tahun 2016 hingga 2018



####7. Tampilkan entry data yang memiliki `duration_time` diatas 4.25 jam. (*Gunakan subquery untuk menjawab pertanyaan ini dengan meng-convert duration_time dari menit ke hour*).

In [45]:
# 1. SELECT * : Memilih keseluruhan kolom dari hasil subquery
# 2. FROM (...) : Menggunakan subquery untuk menambahkan kolom baru, lalu memfilter hasilnya
# 3. SELECT * : Memilih semua kolom dari tabel bike_rent
# 4. (duration_minutes / 60) AS duration_time : Menghitung durasi dalam jam dengan membagi duration_minutes dengan 60 dan menyimpan hasilnya dalam kolom baru, duration_time
# 5. WHERE duration_time > 4.25 : Memfilter hasil hanya untuk baris-baris dengan duration_time lebih besar dari 4.25 jam

df_7 = client.query("""
SELECT *
FROM (
    SELECT *,
    (duration_minutes / 60) AS duration_time
    FROM ftds-hacktiv8-project.practice.bike_rent
    )
WHERE duration_time > 4.25;
""").to_dataframe()

df_7

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,duration_time
0,13255686,Walk Up,539,classic,2016-12-25 11:17:16+00:00,Pease Park,Pease Park,351,5.850000
1,13255691,Walk Up,453,classic,2016-12-25 11:18:17+00:00,Pease Park,Pease Park,350,5.833333
2,14534164,Walk Up,664,classic,2017-04-21 20:46:17+00:00,Davis at Rainey Street,Pease Park,783,13.050000
3,14534142,Walk Up,275,classic,2017-04-21 20:41:54+00:00,Davis at Rainey Street,Pease Park,789,13.150000
4,11965934,Walk Up,453,classic,2016-09-17 11:55:23+00:00,Bullock Museum @ Congress & MLK,Lavaca & 6th,325,5.416667
...,...,...,...,...,...,...,...,...,...
86,14699898,Walk Up,262,classic,2017-05-06 09:08:08+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,350,5.833333
87,9516320,Walk Up,157,classic,2016-04-11 13:19:09+00:00,Lavaca & 6th,State Capitol @ 14th & Colorado,643,10.716667
88,14465838,Walk Up,871,classic,2017-04-16 12:39:51+00:00,South Congress & Academy,State Capitol @ 14th & Colorado,684,11.400000
89,15884138,Walk Up,882,classic,2017-09-07 19:11:26+00:00,Capital Metro HQ - East 5th at Broadway,Capital Metro HQ - East 5th at Broadway,292,4.866667


Tabel di atas merupakan entry data peminjaman dengan durasi meminjam lebih dari **4.25 jam**

### B.) Pandas

### Data Loading

In [46]:
# Load data frame
df = client.query("""
SELECT *
FROM ftds-hacktiv8-project.practice.bike_rent;
""").to_dataframe()

# Menampilkan data frame
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60
...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55


#### 1. Cek kesesuaian tipe data dan keberadaan missing values dengan .info()

In [47]:
# Cek informasi pada tabel
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             8000 non-null   Int64              
 1   subscriber_type     8000 non-null   object             
 2   bike_id             8000 non-null   object             
 3   bike_type           8000 non-null   object             
 4   start_time          8000 non-null   datetime64[us, UTC]
 5   start_station_name  8000 non-null   object             
 6   end_station_name    8000 non-null   object             
 7   duration_minutes    8000 non-null   Int64              
dtypes: Int64(2), datetime64[us, UTC](1), object(5)
memory usage: 515.8+ KB


Berdasarkan informasi di atas, dapat disimpulkan beberapa poin:
*  **Dtype (tipe data)** : Terdapat tipe data yang salah, yaitu tipe data pada kolom `bike_id` seharusnya integer
*   **Missing values** : Tidak terdapat missing values, ditandai dengan seluruh kolom memiliki `8000 non-null` dari total 8000 entries




#### 2. Buat kolom baru bernama quarter

In [48]:
# Membuat kolom quarter
df['quarter'] = pd.to_datetime(df['start_time']).dt.quarter

# Memanggil df
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,quarter
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178,4
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198,1
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60,4
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60,4
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60,4
...,...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55,3
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55,4
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,1
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,2


#### 3. Tampilkan data perjalanan dari tipe member Local31 dengan durasi perjalanan diatas 1 jam

In [49]:
# Filtering 'subscriber_type' dengan nilai 'Local 31' & 'duration_minutes' dengan nilai lebih dari 60 menit
df[(df['subscriber_type'] == ('Local31')) & (df['duration_minutes'] > 60)]

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,quarter


Berdasarkan hasil di atas, **tidak ada data** dari tipe member `Local31` yang melakukan peminjaman sepeda dengan durasi perjalanan lebih dari 1 jam

#### 4. Berapa rata-rata durasi perjalanan dari tipe pelanggan Walk Up dengan tipe sepeda classic pada bulan Maret dari data yang diambil?

In [51]:
# Filter data 'subscriber_type' = 'Walk Up' & 'bike_type' = 'classic' & 'start_time' = 3 ( Maret)
dff_4 = df[ (df['subscriber_type'] == 'Walk Up') &
            (df['bike_type'] == 'classic') &
            (df['start_time'].dt.month == 3)]

# Mean (rata-rata) 'duration_minutes' dari data yang sudah difilter
dff_4['duration_minutes'].mean()

38.476584022038566

Mean (rata-rata) durasi perjalanan dari subscriber tipe `Walk Up` yang meminjam sepeda tipe `classic` pada bulan `Maret`, melakukan peminjaman selama **38.47 menit**